In [ ]:
#######################################
## SIS 420                           ##
## Alumno: Sanchez Calvimontes Pablo ##
## Carrera: Ingenieria de Sistemas   ##
#######################################

# se uso el dataset de:
# https://www.kaggle.com/datasets/itsnahm/data-house

# en este data set podemos ver que existen columnas que no son utiles
# asi que se pocedio a eliminarlas
# las columnas "id", "unnamed"  fueron las columnas que se eliminaron

# este data set tiene orgininalmente 22 columnas
# unnamed
#id
# date
# price
# bedrooms
# bathrooms
# sqft_living
# sqft_lot
# floors
# waterfront
# view
# condition
# grade
# sqft_above
# sqft_basement
# yr_built
# yr_renovated
# zipcode
# lat
# long
# sqft_living15
# sqft_lot15


In [ ]:
import random
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [ ]:
class MLP:
    def __init__(self, layers):
        # el MLP es una lista de capas
        self.layers = layers

    def __call__(self, x):
        # calculamos la salida del modelo aplicando
        # cada capa de manera secuencial
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
class Layer():
    def __init__(self):
        self.params = []
        self.grads = []

    def __call__(self, x):
        # por defecto, devolver los inputs
        # cada capa hará algo diferente aquí
        return x

    def backward(self, grad):
        # cada capa, calculará sus gradientes
        # y los devolverá para las capas siguientes
        return grad

    def update(self, params):
        # si hay parámetros, los actualizaremos
        # con lo que nos de el optimizer
        return

In [ ]:
class Linear(Layer):
    def __init__(self, d_in, d_out):
        # pesos de la capa
        self.w = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(d_in+d_out)),
                                  size=(d_in, d_out))
        self.b = np.zeros(d_out)

    def __call__(self, x):
        self.x = x
        self.params = [self.w, self.b]
        # salida del preceptrón
        return np.dot(x, self.w) + self.b

    def backward(self, grad_output):
        # gradientes para la capa siguiente (BACKPROP)
        grad = np.dot(grad_output, self.w.T)
        self.grad_w = np.dot(self.x.T, grad_output)
        # gradientes para actualizar pesos
        self.grad_b = grad_output.mean(axis=0)*self.x.shape[0]
        self.grads = [self.grad_w, self.grad_b]
        return grad

    def update(self, params):
        self.w, self.b = params

In [ ]:
#funciones de activacion
class ReLU(Layer):
    def __call__(self, x):
        self.x = x
        return np.maximum(0, x)

    def backward(self, grad_output):
        grad = self.x > 0
        return grad_output*grad

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

class Sigmoid(Layer):
    def __call__(self, x):
        self.x = x
        return sigmoid(x)

    def backward(self, grad_output):
        grad = sigmoid(self.x)*(1 - sigmoid(self.x))
        return grad_output*grad

In [ ]:
#Optimizador, stocastic greadent descent
class SGD():
    def __init__(self, net, lr):
        self.net = net
        self.lr = lr

    def update(self):
        for layer in self.net.layers:
            layer.update([
                params - self.lr*grads
                for params, grads in zip(layer.params, layer.grads)
            ])

In [ ]:
class Loss():
    def __init__(self, net):
        self.net = net

    def backward(self):
        # derivada de la loss function con respecto
        # a la salida del MLP
        grad = self.grad_loss()
        # BACKPROPAGATION
        for layer in reversed(self.net.layers):
            grad = layer.backward(grad)

class MSE(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target.reshape(output.shape)
        loss = np.mean((self.output - self.target)**2)
        return loss.mean()

    def grad_loss(self):
        return self.output -  self.target

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

#Carga de dataset
data = pd.read_csv('/content/drive/MyDrive/EXAMEN_FINAL_SanchezPablo/ej2/Datasets/data_house.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     21613 non-null  int64  
 1   id             21613 non-null  int64  
 2   date           21613 non-null  object 
 3   price          21613 non-null  float64
 4   bedrooms       21613 non-null  int64  
 5   bathrooms      21613 non-null  float64
 6   sqft_living    21613 non-null  int64  
 7   sqft_lot       21613 non-null  int64  
 8   floors         21613 non-null  float64
 9   waterfront     21613 non-null  int64  
 10  view           21613 non-null  int64  
 11  condition      21613 non-null  int64  
 12  grade          21613 non-null  int64  
 13  sqft_above     21613 non-null  int64  
 14  sqft_basement  21613 non-null  int64  
 15  yr_built       21613 non-null  int64  
 16  yr_renovated   21613 non-null  int64  
 17  zipcode        21613 non-null  int64  
 18  lat   

In [ ]:
columnas_categoricas = data.select_dtypes(include=['object']).columns
columnas_fechas = data.select_dtypes(include=['datetime64']).columns

#Procesamiento de frases
for columna in columnas_categoricas:
  le = LabelEncoder()
  data[columna] = le.fit_transform(data[columna])

#Fechas a numeros
for columna in columnas_fechas:
  data[columna] = pd.to_numeric(data[columna].map(datetime.timestamp))

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     21613 non-null  int64  
 1   id             21613 non-null  int64  
 2   date           21613 non-null  int64  
 3   price          21613 non-null  float64
 4   bedrooms       21613 non-null  int64  
 5   bathrooms      21613 non-null  float64
 6   sqft_living    21613 non-null  int64  
 7   sqft_lot       21613 non-null  int64  
 8   floors         21613 non-null  float64
 9   waterfront     21613 non-null  int64  
 10  view           21613 non-null  int64  
 11  condition      21613 non-null  int64  
 12  grade          21613 non-null  int64  
 13  sqft_above     21613 non-null  int64  
 14  sqft_basement  21613 non-null  int64  
 15  yr_built       21613 non-null  int64  
 16  yr_renovated   21613 non-null  int64  
 17  zipcode        21613 non-null  int64  
 18  lat   

In [ ]:
columnas_eiliminar = ['Unnamed: 0', 'id', 'date']
data = data.drop(columnas_eiliminar, axis=1)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          21613 non-null  float64
 1   bedrooms       21613 non-null  int64  
 2   bathrooms      21613 non-null  float64
 3   sqft_living    21613 non-null  int64  
 4   sqft_lot       21613 non-null  int64  
 5   floors         21613 non-null  float64
 6   waterfront     21613 non-null  int64  
 7   view           21613 non-null  int64  
 8   condition      21613 non-null  int64  
 9   grade          21613 non-null  int64  
 10  sqft_above     21613 non-null  int64  
 11  sqft_basement  21613 non-null  int64  
 12  yr_built       21613 non-null  int64  
 13  yr_renovated   21613 non-null  int64  
 14  zipcode        21613 non-null  int64  
 15  lat            21613 non-null  float64
 16  long           21613 non-null  float64
 17  sqft_living15  21613 non-null  int64  
 18  sqft_l

In [ ]:
print(data)

          price  bedrooms  bathrooms  sqft_living  sqft_lot  floors  \
0      221900.0         3       1.00         1180      5650     1.0   
1      538000.0         3       2.25         2570      7242     2.0   
2      180000.0         2       1.00          770     10000     1.0   
3      604000.0         4       3.00         1960      5000     1.0   
4      510000.0         3       2.00         1680      8080     1.0   
...         ...       ...        ...          ...       ...     ...   
21608  360000.0         3       2.50         1530      1131     3.0   
21609  400000.0         4       2.50         2310      5813     2.0   
21610  402101.0         2       0.75         1020      1350     2.0   
21611  400000.0         3       2.50         1600      2388     2.0   
21612  325000.0         2       0.75         1020      1076     2.0   

       waterfront  view  condition  grade  sqft_above  sqft_basement  \
0               0     0          3      7        1180              0   
1  

In [ ]:
x = data.iloc[:, 1:18]
y = data.iloc[:, 0]
m = y.size
print(x.shape)
print(y.shape)
print(x)
print('---'*20)
print(y)

(21613, 17)
(21613,)
       bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  view  \
0             3       1.00         1180      5650     1.0           0     0   
1             3       2.25         2570      7242     2.0           0     0   
2             2       1.00          770     10000     1.0           0     0   
3             4       3.00         1960      5000     1.0           0     0   
4             3       2.00         1680      8080     1.0           0     0   
...         ...        ...          ...       ...     ...         ...   ...   
21608         3       2.50         1530      1131     3.0           0     0   
21609         4       2.50         2310      5813     2.0           0     0   
21610         2       0.75         1020      1350     2.0           0     0   
21611         3       2.50         1600      2388     2.0           0     0   
21612         2       0.75         1020      1076     2.0           0     0   

       condition  grade  sqft_

In [ ]:
# Crea un nuevo DataFrame con los datos modificados
nuevo_data = data.copy()
# Guardar el dataset actualizado en un nuevo archivo
nuevo_data.to_csv('data_house2.csv', index=False)

In [ ]:
data = np.loadtxt("/content/data_house2.csv", delimiter=',',skiprows=1)
# print(data)
x, y = data[:, 1:18].astype(int), data[:, 0].astype(int)
x = x.reshape(len(x),17)
print(x.shape)
print(y.shape)

(21613, 17)
(21613,)


In [ ]:
x_train, x_test, y_train, y_test = x[:800] , x[800:] , y[:800].astype(np.int64), y[800:].astype(np.int64)
x_train.shape , x_test.shape

y_train.shape , y_test.shape

((800,), (20813,))

In [ ]:
def relu(x):
  return np.maximum(0, x)

def reluPrime(x):
  return x > 0

In [ ]:
def linear(x):
    return x

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

In [ ]:
# Mean Square Error -> usada para regresión (con activación lineal)
def mse(y, y_hat):
    return np.mean((y_hat - y.reshape(y_hat.shape))**2)


In [ ]:
def grad_mse(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)


In [ ]:
# clase base MLP

class MLP():
  def __init__(self, D_in, H, D_out, loss, grad_loss, activation):
    # pesos de la capa 1
    self.w1, self.b1 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(D_in+H)),
                                  size=(D_in, H)), np.zeros(H)
    # pesos de la capa 2
    self.w2, self.b2 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(H+D_out)),
                                  size=(H, D_out)), np.zeros(D_out)
    self.ws = []
    # función de pérdida y derivada
    self.loss = loss
    self.grad_loss = grad_loss
    # función de activación
    self.activation = activation

  def __call__(self, x):
    # salida de la capa 1
    self.h_pre = np.dot(x, self.w1) + self.b1
    self.h = relu(self.h_pre)
    # salida del MLP
    y_hat = np.dot(self.h, self.w2) + self.b2
    return self.activation(y_hat)

  def fit(self, X, Y, epochs = 100, lr = 0.001, batch_size=None, verbose=True, log_each=1):
    batch_size = len(X) if batch_size == None else batch_size
    batches = len(X) // batch_size
    l = []
    for e in range(1,epochs+1):
        # Mini-Batch Gradient Descent
        _l = []
        for b in range(batches):
            # batch de datos
            x = X[b*batch_size:(b+1)*batch_size]
            y = Y[b*batch_size:(b+1)*batch_size]
            # salida del perceptrón
            y_pred = self(x)
            # función de pérdida
            loss = self.loss(y, y_pred)
            _l.append(loss)
            # Backprop
            dldy = self.grad_loss(y, y_pred)
            grad_w2 = np.dot(self.h.T, dldy)
            grad_b2 = dldy.mean(axis=0)
            dldh = np.dot(dldy, self.w2.T)*reluPrime(self.h_pre)
            grad_w1 = np.dot(x.T, dldh)
            grad_b1 = dldh.mean(axis=0)
            # Update (GD)
            self.w1 = self.w1 - lr * grad_w1
            self.b1 = self.b1 - lr * grad_b1
            self.w2 = self.w2 - lr * grad_w2
            self.b2 = self.b2 - lr * grad_b2
        l.append(np.mean(_l))
        # guardamos pesos intermedios para visualización
        self.ws.append((
            self.w1.copy(),
            self.b1.copy(),
            self.w2.copy(),
            self.b2.copy()
        ))
        if verbose and not e % log_each:
            print(f'Epoch: {e}/{epochs}, Loss: {np.mean(l):.5f}')

  def predict(self, ws, x):
    w1, b1, w2, b2 = ws
    h = relu(np.dot(x, w1) + b1)
    y_hat = np.dot(h, w2) + b2
    return self.activation(y_hat)

In [ ]:
# MLP para regresión
class MLPRegression(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, mse, grad_mse, linear)


In [ ]:
model = MLPRegression(D_in=17, H=100, D_out=1)
epochs, lr = 100, 0.02

# normalización datos
x_mean = np.mean(x_train)
x_std = np.std(x_train)
x_std = np.nan_to_num(x_std, nan=1.0)


# Calcula x_norm
x_norm = (x_train - x_mean) / x_std

model.fit(x_norm, y_train, epochs, lr, batch_size=1, log_each=10)

Epoch: 10/100, Loss: 146801895223916556227167441905737277310208709496398265809597424728036286329257984.00000
Epoch: 20/100, Loss: 73400947611958278113583720952868638655104354748199132904798712364018143164628992.00000
Epoch: 30/100, Loss: 48933965074638847687710327778696313356818416345864741982570166553132459846795264.00000
Epoch: 40/100, Loss: 36700473805979139056791860476434319327552177374099566452399356182009071582314496.00000
Epoch: 50/100, Loss: 29360379044783309270828019595700204876078722830459047182636446185311421247520768.00000
Epoch: 60/100, Loss: 24466982537319423843855163889348156678409208172932370991285083276566229923397632.00000
Epoch: 70/100, Loss: 20971699317702363764877156854071574911484802021756462273311747275222443748229120.00000
Epoch: 80/100, Loss: 18350236902989569528395930238217159663776088687049783226199678091004535791157248.00000
Epoch: 90/100, Loss: 16311321691546281465567071045428076348960017077054577339032811761990911049859072.00000
Epoch: 100/100, Loss: 14680

In [ ]:
# últimos pesos encontrados

w = model.ws[-1]
w

In [ ]:
print("Primeros 5 datos de entrada de prueba (X_test):")
print(x_test[:5])

print("Primeras 5 etiquetas de prueba (y_test):")
print(y_test[:5])

Primeros 5 datos de entrada de prueba (X_test):
[[    3     2  1340  2720     1     0     0     3     7  1340     0  1913
      0 98103    47  -122  2030]
 [    4     2  3190  8684     1     0     3     5     9  1680  1510  1967
      0 98006    47  -122  3160]
 [    3     2  1630  4534     2     0     0     3     8  1630     0  1987
      0 98034    47  -122  1380]
 [    5     2  2990 15085     2     0     0     3     9  2990     0  2007
      0 98011    47  -122  3150]
 [    4     1  1550  7740     1     0     0     3     6  1550     0  1954
      0 98126    47  -122  1220]]
Primeras 5 etiquetas de prueba (y_test):
[ 650000 1216000  422120  625000  385200]


In [ ]:
# nuevo punto
x_new = x_test
X_new = [3, 2, 1340, 2720, 1, 0, 0, 3, 7, 1340, 0, 1913, 0, 98103, 47, -122, 2030]
y_pred = model.predict(w, X_new)
y_pred

array([546724.04506909])

In [ ]:
def evaluate(perceptron, x, t = 0.5):
    w = perceptron.ws[-1]
    x = np.c_[np.ones(len(x)), x]
    y = perceptron(w, x)
    return (y > t).astype(np.int64)

In [ ]:
def accuracy (y_pred, y):
  return np.sum(y_pred == y_test) / len(str(y))

In [ ]:
print(accuracy(650000, 546724.04)/100)

0.11777777777777779
